In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
#set the proxy so the geopy library can use it to visit external web sites
# @hidden_cell
proxy_addr = '135.245.48.34:8000'
os.environ['http_proxy'] = proxy_addr
os.environ['https_proxy'] = proxy_addr

Use the pandas read_html() to read the tables from the HTML page referenced by the url 
The table is the only one on this HTML page so it is the first one in the list; 

In [4]:
df_pcc = pd.read_html(url)[0]
df_pcc

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
...,...,...,...
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...


As requested, Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned. 
Remove those rows with Borough = "Not assigned"

In [5]:
noAssign_idx = df_pcc[df_pcc['Borough']=='Not assigned'].index
df_pcc.drop(noAssign_idx,inplace=True)
df_pcc

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
...,...,...,...
160,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing CentrE
169,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


from the following test code, it looks there are no more than duplicated entries of 'Postal code' exist in one postal code area

In [6]:
len(df_pcc['Postal code'].unique()) == df_pcc.shape[0]

True

As requested, if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
After following code check, there is no such record

In [7]:
df_pcc[df_pcc['Neighborhood']=='Not assigned'].shape[0]

0

In [8]:
df_pcc.shape

(103, 3)

Read thegeographical coordinates code from the csv file, as the google data read is not stable

In [9]:
df_geocode=pd.read_csv('http://cocl.us/Geospatial_data')
df_geocode

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [10]:
df_pcc.reset_index(drop=True, inplace=True)

In [11]:
# change the column name to be the same of two dataframe so that they can merge next
df_geocode.rename(columns={'Postal Code': 'Postal code'}, inplace=True)

In [12]:
# merge the two dataframe with the same value in column 'Postal code'
pd.merge(df_pcc, df_geocode, on='Postal code')

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North,43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...,43.636258,-79.498509
